In [ ]:
!pip -q install --upgrade ffmpeg-python \
                      git+https://github.com/openai/whisper.git \
                      soundfile datasets


!pip -q install --upgrade yt_dlp


import os, json, subprocess, uuid, torch, whisper, glob
from pathlib import Path

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import shutil
import nltk

# Step 1: Delete any corrupted punkt folders manually
shutil.rmtree('/root/nltk_data/tokenizers/punkt', ignore_errors=True)
shutil.rmtree('/usr/local/share/nltk_data/tokenizers/punkt', ignore_errors=True)

# Step 2: Force a clean re-download
nltk.download('punkt', download_dir='/usr/local/share/nltk_data')

[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
MODEL_SIZE = "turbo"
DEVICE ="cuda" if torch.cuda.is_available() else "cpu"

AUDIO_DIR = Path("/content/content/audio")
AUDIO_DIR.mkdir(exist_ok=True, parents=True)

print(f"using {MODEL_SIZE} model on device {DEVICE}")

using turbo model on device cuda


In [ ]:
import yt_dlp

# List of YouTube links to download
youtube_links = [
    "https://youtu.be/3iqvBEGS2So"

]

def download_youtube_audio_m4a(urls, output_dir):
    ydl_opts = {
        'format': 'bestaudio[ext=m4a]/bestaudio/best',
        'outtmpl': str(output_dir / '%(title)s.%(ext)s'),
        'quiet': True,
        'noplaylist': True,
        # No postprocessor to avoid converting to mp3
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        for url in urls:
            try:
                print(f"Downloading: {url}")
                ydl.download([url])
            except Exception as e:
                print(f"Failed to download {url}: {e}")

# Run the function
download_youtube_audio_m4a(youtube_links, AUDIO_DIR)


Downloading: https://youtu.be/3iqvBEGS2So


In [ ]:
import subprocess

# Choose output format: "wav" or "m4a"
OUTPUT_FORMAT = "wav"  # change to "m4a" if preferred
OUTPUT_DIR = AUDIO_DIR / "cleaned"
OUTPUT_DIR.mkdir(exist_ok=True)

def clean_audio_file(input_path, output_path):
    command = [
        "ffmpeg", "-y", "-i", str(input_path),
        "-af", (
            "highpass=f=80,"
            "afftdn,"
            "loudnorm=I=-16:LRA=11:tp=-1.5,"
            "dynaudnorm=f=200,"
            "apad=pad_dur=0.1"
        ),
        "-ar", "24000",        # Set sample rate to 24kHz
        "-ac", "1",            # Mono audio
        str(output_path)
    ]

    try:
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"Cleaned: {input_path.name}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to clean {input_path.name}: {e}")

# Loop through all downloaded .m4a files
for audio_file in AUDIO_DIR.glob("*.m4a"):
    output_file = OUTPUT_DIR / (audio_file.stem + f"_cleaned.{OUTPUT_FORMAT}")
    clean_audio_file(audio_file, output_file)


Cleaned: Fine-tune Orpheus or Sesame CSM-1B with Unsloth (Voice Cloning Tutorial).m4a


In [ ]:
from IPython.display import Audio, display

def extract_preview(input_path, output_path, duration=10):
    command = [
        "ffmpeg", "-y",
        "-i", str(input_path),
        "-t", str(duration),
        "-ar", "24000",  # Ensure same sample rate
        "-ac", "1",      # Mono
        str(output_path)
    ]
    subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Preview directory
PREVIEW_DIR = AUDIO_DIR / "previews"
PREVIEW_DIR.mkdir(exist_ok=True)

# Process and play previews
for original_file in AUDIO_DIR.glob("*.m4a"):
    cleaned_file = AUDIO_DIR / "cleaned" / (original_file.stem + "_cleaned.wav")

    # Define preview paths
    original_preview = PREVIEW_DIR / (original_file.stem + "_orig_10s.wav")
    cleaned_preview = PREVIEW_DIR / (original_file.stem + "_clean_10s.wav")

    # Extract first 10 seconds
    extract_preview(original_file, original_preview)
    extract_preview(cleaned_file, cleaned_preview)

    # Display audio players
    print(f"🎧 Preview: {original_file.stem}")
    print("Original Audio:")
    display(Audio(str(original_preview)))
    print("Cleaned Audio:")
    display(Audio(str(cleaned_preview)))


🎧 Preview: Fine-tune Orpheus or Sesame CSM-1B with Unsloth (Voice Cloning Tutorial)
Original Audio:


Cleaned Audio:


In [ ]:
from pathlib import Path
import whisper
import torch
import json

# Load the model
model = whisper.load_model(MODEL_SIZE, device=DEVICE)
use_fp16 = DEVICE == "cuda"

# Paths
CLEANED_DIR = AUDIO_DIR / "cleaned"
TRANSCRIPTS_JSON_DIR = AUDIO_DIR / "json_transcripts"
TRANSCRIPTS_JSON_DIR.mkdir(exist_ok=True)

def transcribe_to_json(audio_path, output_dir):
    result = model.transcribe(
        str(audio_path),
        word_timestamps=True,
        fp16=use_fp16,
        verbose=False
    )

    # Add audio path to the result
    result["audio_path"] = str(audio_path)

    # Save result as JSON
    output_path = output_dir / (audio_path.stem + ".json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    print(f"✅ Saved JSON: {output_path.name}")

# Run transcription
for audio_file in CLEANED_DIR.glob("*.wav"):
    transcribe_to_json(audio_file, TRANSCRIPTS_JSON_DIR)


Detected language: English


100%|██████████| 352741/352741 [03:32<00:00, 1657.78frames/s]

✅ Saved JSON: Fine-tune Orpheus or Sesame CSM-1B with Unsloth (Voice Cloning Tutorial)_cleaned.json


In [ ]:
import json
from pathlib import Path

# Paths (use the same as your original code)
json_path = Path("/content/content/audio/json_transcripts/Fine-tune Orpheus or Sesame CSM-1B with Unsloth (Voice Cloning Tutorial)_cleaned.json")
output_dir = Path("dataset_30s")
output_dir.mkdir(exist_ok=True, parents=True)

# Load JSON data
with open(json_path, "r") as f:
    data = json.load(f)

segments = data.get("segments", [])

# Create list of sentences with start/end times
sentences = []
for seg in segments:
    sentences.append({
        "start": seg["start"],
        "end": seg["end"],
        "text": seg["text"].strip()
    })

# Save to a new JSON file
sentences_path = output_dir / "sentences.json"
with open(sentences_path, "w") as f:
    json.dump(sentences, f, indent=2)

print(f"✅ Saved {len(sentences)} sentences with timings to '{sentences_path}'")



✅ Saved 890 sentences with timings to 'dataset_30s/sentences.json'


In [ ]:
import json
from pathlib import Path
from pydub import AudioSegment

# Paths (update these to your actual files)
json_path = Path("/content/dataset_30s/sentences.json")
audio_path = Path("/content/content/audio/cleaned/Fine-tune Orpheus or Sesame CSM-1B with Unsloth (Voice Cloning Tutorial)_cleaned.wav")
output_dir = Path("dataset_final")
output_dir.mkdir(exist_ok=True, parents=True)

# Load segments JSON
with open(json_path, "r") as f:
    data = json.load(f)

segments = data

MAX_CHUNK_DURATION = 30.0  # seconds

chunks = []
current_chunk = {
    "segments": [],
    "start": None,
    "end": None
}

for seg in segments:
    seg_start = seg["start"]
    seg_end = seg["end"]

    # If chunk empty, start new chunk
    if current_chunk["start"] is None:
        current_chunk["start"] = seg_start
        current_chunk["end"] = seg_end
        current_chunk["segments"].append(seg)
        continue

    # Calculate duration if this segment is added
    prospective_end = seg_end
    duration = prospective_end - current_chunk["start"]

    if duration <= MAX_CHUNK_DURATION:
        # Add segment to current chunk
        current_chunk["segments"].append(seg)
        current_chunk["end"] = seg_end
    else:
        # Current chunk full, save it
        chunks.append(current_chunk)
        # Start new chunk with current segment
        current_chunk = {
            "segments": [seg],
            "start": seg_start,
            "end": seg_end
        }

# Add last chunk
if current_chunk["segments"]:
    chunks.append(current_chunk)

# Load full audio file
audio = AudioSegment.from_file(audio_path)

# Export chunks and save metadata
metadata = []

for i, chunk in enumerate(chunks):
    start_ms = int(chunk["start"] * 1000)
    end_ms = int(chunk["end"] * 1000)
    clip = audio[start_ms:end_ms]

    fname = f"chunk_{i:04}.wav"
    clip.export(output_dir / fname, format="wav")

    # Concatenate texts from all segments in the chunk
    chunk_text = " ".join(seg["text"].strip() for seg in chunk["segments"])

    metadata.append({
        "file": fname,
        "start": chunk["start"],
        "end": chunk["end"],
        "text": chunk_text
    })

# Save metadata JSON
with open(output_dir / "metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Created {len(chunks)} audio chunks and metadata at '{output_dir}'")


✅ Created 125 audio chunks and metadata at 'dataset_final'


In [ ]:
!pip install torchcodec

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import json
import os
from pydub import AudioSegment
from datasets import Dataset, Audio

# Load transcript json
with open("/content/dataset_30s/sentences.json") as f:
    transcript = json.load(f)

# Load audio
audio = AudioSegment.from_file("/content/content/audio/cleaned/Fine-tune Orpheus or Sesame CSM-1B with Unsloth (Voice Cloning Tutorial)_cleaned.wav")

os.makedirs("audio_chunks_24000", exist_ok=True)

chunk_duration_limit = 30.0  # max chunk length in seconds
chunks = []

current_chunk = []
current_start = None
current_end = None

for segment in transcript:
    start, end = segment["start"], segment["end"]

    # If starting a new chunk
    if current_start is None:
        current_start = start

    # Check if adding this segment exceeds chunk duration limit
    if current_end is not None and (end - current_start) > chunk_duration_limit:
        # Export audio chunk
        chunk_audio = audio[int(current_start * 1000):int(current_end * 1000)]
        filename = f"audio_chunks/chunk_{len(chunks):04d}.wav"
        chunk_audio.export(filename, format="wav")

        # Add chunk entry
        chunks.append({
            "audio": filename,
            "transcription": " ".join([s["text"] for s in current_chunk])
        })

        # Start a new chunk
        current_chunk = [segment]
        current_start = start
        current_end = end
    else:
        # Continue current chunk
        current_chunk.append(segment)
        current_end = end

# Save the last chunk
if current_chunk:
    chunk_audio = audio[int(current_start * 1000):int(current_end * 1000)]
    filename = f"audio_chunks/chunk_{len(chunks):04d}.wav"
    chunk_audio.export(filename, format="wav")

    chunks.append({
        "audio": filename,
        "transcription": " ".join([s["text"] for s in current_chunk])
    })

# Create Hugging Face Dataset
dataset = Dataset.from_list(chunks)

# Define the audio feature to load audio files automatically
dataset = dataset.cast_column("audio", Audio(sampling_rate=24000))

# Save locally for upload
dataset.save_to_disk("voice_cloning_dataset_24000")

print("Dataset prepared! You can now upload it to Hugging Face.")


Saving the dataset (0/1 shards):   0%|          | 0/125 [00:00<?, ? examples/s]

Dataset prepared! You can now upload it to Hugging Face.


In [ ]:
!pip install torchcodec
from datasets import load_dataset
from IPython.display import Audio

# Replace this with your actual dataset repo name on Hugging Face
dataset_repo = "gary7372/voice_clone_sample_24000hz"

# Load the dataset (this returns a DatasetDict with splits)
dataset = load_dataset(dataset_repo)

# See available splits
print("Available splits:", dataset.keys())

# Select the 'train' split (or replace with the correct split name)
train_dataset = dataset["train"]

# Check first 3 samples
for i in range(3):
    sample = train_dataset[i]
    print(f"Sample {i} transcription:")
    print(sample["transcription"])
    print(f"Audio file or array info: {sample['audio']}")
    print()

# Play the audio of the first sample
audio_sample = train_dataset[0]["audio"]

print(f"Sampling rate: {audio_sample['sampling_rate']}")
print(f"Audio array shape: {audio_sample['array'].shape}")

display(Audio(audio_sample["array"], rate=audio_sample["sampling_rate"]))


Available splits: dict_keys(['train'])


ImportError: To support decoding audio data, please install 'torchcodec'.